# How to connect to OpenAI service via PromptSail and OpenAI Python SDK

First import the OpenAI Python SDK and load your API key from the environment.


Install all the necessary packages from [examples/pyproject.toml](pyproject.toml) by running the following command:

```bash 
cd prompt_sail/examples
poetry install
```

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, dotenv_values
from rich import print

config = dotenv_values(".env")

openai_key = config["OPENAI_API_KEY"]
openai_org_id = config["OPENAI_ORG_ID"]
print(
    f"OpenAI api key={openai_key[0:3]}...{openai_key[-3:]}"
)

OpenAI api key=sk-...fRh

Lets see what gpt models are available for the API key we loaded.

In [2]:
client_org = OpenAI(
    organization=openai_org_id,
    api_key=openai_key,
)
models = client_org.models.list().data
models = [m for m in models if "gpt" in m.id.lower()]

# sort models by model id
models = sorted(models, key=lambda x: x.id)

# print models which contains gpt in the name
print(models)

[
    Model(id='gpt-3.5-turbo', created=1677610602, object='model', owned_by='openai'),
    Model(id='gpt-3.5-turbo-0125', created=1706048358, object='model', owned_by='system'),
    Model(id='gpt-3.5-turbo-0301', created=1677649963, object='model', owned_by='openai'),
    Model(id='gpt-3.5-turbo-0613', created=1686587434, object='model', owned_by='openai'),
    Model(id='gpt-3.5-turbo-1106', created=1698959748, object='model', owned_by='system'),
    Model(id='gpt-3.5-turbo-16k', created=1683758102, object='model', owned_by='openai-internal'),
    Model(id='gpt-3.5-turbo-16k-0613', created=1685474247, object='model', owned_by='openai'),
    Model(id='gpt-3.5-turbo-instruct', created=1692901427, object='model', owned_by='system'),
    Model(id='gpt-3.5-turbo-instruct-0914', created=1694122472, object='model', owned_by='system'),
    Model(id='gpt-4', created=1687882411, object='model', owned_by='openai'),
    Model(id='gpt-4-0125-preview', created=1706037612, object='model', owned_by='system'),
    Model(id='gpt-4-0613', created=1686588896, object='model', owned_by='openai'),
    Model(id='gpt-4-1106-preview', created=1698957206, object='model', owned_by='system'),
    Model(id='gpt-4-1106-vision-preview', created=1711473033, object='model', owned_by='system'),
    Model(id='gpt-4-turbo', created=1712361441, object='model', owned_by='system'),
    Model(id='gpt-4-turbo-2024-04-09', created=1712601677, object='model', owned_by='system'),
    Model(id='gpt-4-turbo-preview', created=1706037777, object='model', owned_by='system'),
    Model(id='gpt-4-vision-preview', created=1698894917, object='model', owned_by='system'),
    Model(id='gpt-4o', created=1715367049, object='model', owned_by='system'),
    Model(id='gpt-4o-2024-05-13', created=1715368132, object='model', owned_by='system')
]

Test the direct connection to OpenAI service via OpenAI Python SDK.

In [3]:
haiku_prompt = [
    {
        "role": "system",
        "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair.",
    },
    {
        "role": "user",
        "content": "Compose a haiku that explains the concept of recursion in programming.",
    },
]

poem_prompt = [
    {
        "role": "system",
        "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair.",
    },
    {
        "role": "user",
        "content": "Compose a five line poem that explains the concept of recursion in programming.",
    },
]

yoda_prompt = [
    {
        "role": "system",
        "content": "Yoda assistant you are, skilled in explaining complex life and phisopical matters.",
    },
    {
        "role": "user",
        "content": "What number 42 means, be brief.",
    },
]


teacher_chat=[
        {
            "role": "system",
            "content": "You are teacher and help studends learn phisics, geography and chemistry.",
        },
        {
            "role": "user",
            "content": "Name the plantets in solar system.",
        },
        {
            "role": "assistant",
            "content": "The planets in our solar system, listed in order from closest to the Sun to farthest, are:\n\n1. Mercury\n2. Venus\n3. Earth\n4. Mars\n5. Jupiter\n6. Saturn\n7. Uranus\n8. Neptune\n\nIt is worth mentioning that there is ongoing debate about the classification of Pluto, which was previously considered the ninth planet but is now classified as a dwarf planet.",        
        }, 
        { 
            "role": "user",
            "content": "What is the distance between Earth and Mars?"
        }
]

In [4]:
oai_client = OpenAI(api_key=openai_key)

response = oai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=teacher_chat,
)

print(response.choices[0].message.content)

The distance between Earth and Mars varies depending on the positions of the two planets in their respective orbits
around the Sun. On average, the distance between Earth and Mars is about 225 million kilometers (140 million 
miles). However, because both planets have elliptical orbits, the actual distance can range from about 54.6 million
kilometers (33.9 million miles) at their closest approach (opposition) to about 401 million kilometers (249 million
miles) at their farthest apart (conjunction).

Testing the streaming API

In [5]:
oai_client = OpenAI(api_key=openai_key)

response = oai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    stream=True,
    messages=haiku_prompt
)

for chunk in response:
    print(chunk.choices[0].delta.content)

In

code

dance

,

loop

...

leaves

Iterations

thrive

.

None

## Create a request to the OpenAI via promptsail proxy

Run the docker and go to PromptSail UI http://localhost/


At the application start we created test projects (models-playground) with OpenAI API deployment. We will use project1 for this example.

In this case we will use the default `Models Playground` settings:
* with project_slug -> 'models-playground' 
* deployment_name -> 'openai'
resulting in promptsail proxy url like this: 

**http://localhost:8000/models-playground/openai/** -> https://api.openai.com/v1

You can create your own project if you want.



In [6]:
ps_api_base = "http://localhost:8000/models-playground/openai/"

# adress with tags 
#ps_api_base = "http://localhost:8000/models-playground/openai/?tags=examples,openai_sdk,chat,user_ks&target_path="


ps_client = OpenAI(base_url=ps_api_base, api_key=openai_key, max_retries=0)


# model_name = "gpt-4-turbo-preview"
model_name = "gpt-3.5-turbo-0125"

response = ps_client.chat.completions.create(
    model=model_name,
    temperature=0.5,
    messages=teacher_chat,
)

print(response.choices[0].message.content)

The distance between Earth and Mars varies depending on their positions in their respective orbits around the Sun. 
On average, the distance between Earth and Mars is about 225 million kilometers (140 million miles). However, 
because both planets have elliptical orbits, this distance can range from about 54.6 million kilometers (33.9 
million miles) at their closest approach (opposition) to about 401 million kilometers (249 million miles) at their 
farthest distance (conjunction).

### Streaming API

In [7]:
response = ps_client.chat.completions.create(
    model="gpt-3.5-turbo",
    stream=True,
    messages=haiku_prompt
)

for chunk in response:
    print(chunk.choices[0].delta.content)

In

loops

we

wander

,

...

less

echoes

code

.

None